In [1]:
import pandas as pd
from tqdm import tqdm
import torch
import pytorch_utils
DATA_DIR = 'data/'

[PyTorch-Utils]: Loading anchor candidate data...
[PyTorch-Utils]: Loading wikipedia title embedings...
[PyTorch-Utils]: Loading wikipedia items...


In [2]:
# is cuda available?
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
dataset = pytorch_utils.EntityDataset(device='cpu')
test_dataset = pytorch_utils.EntityDataset(train=False, device='cpu')

[PyTorch-Utils]: Loading train set...


c:\Users\aybar\Documents\CS423-Project-3\pytorch_utils.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.train_data_no_doc_start['entity_loc'] = self.train_data_no_doc_start.groupby(['doc_id']).cumcount()
100%|██████████| 18288/18288 [02:17<00:00, 133.03it/s]


[PyTorch-Utils]: Saving train_context_text_150.pkl to data/pkl...
[PyTorch-Utils]: Now generating entity embeddings...


Batches:   0%|          | 0/572 [00:00<?, ?it/s]

Entity Length: 18288
Entity shape: torch.Size([18288, 384])
[PyTorch-Utils]: Loading test set...


c:\Users\aybar\Documents\CS423-Project-3\pytorch_utils.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.train_data_no_doc_start['entity_loc'] = self.train_data_no_doc_start.groupby(['doc_id']).cumcount()
100%|██████████| 9166/9166 [00:38<00:00, 240.57it/s]


[PyTorch-Utils]: Saving test_context_text_150.pkl to data/pkl...
[PyTorch-Utils]: Now generating entity embeddings...


Batches:   0%|          | 0/287 [00:00<?, ?it/s]

Entity Length: 9166
Entity shape: torch.Size([9166, 384])


In [ ]:
del pytorch_utils.corpus_embeddings

In [4]:
from torch.utils.data import DataLoader

dataloader = DataLoader(dataset, batch_size=16, shuffle=True, collate_fn=lambda x: pytorch_utils.EntityDataset.collate_fn_train(x, device=device, entity_embed_tensor=dataset.entity_embeddings))
# test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False, collate_fn=lambda x: pytorch_utils.EntityDataset.collate_fn_test(x, device=device))

In [5]:
model = pytorch_utils.EntityClassifier(transformer_model='bert-base-uncased', hidden_size=256, device=device)

In [6]:
"""
Freeze everything except for:
transformer.encoder.layer.11.attention.self.query.weight
transformer.encoder.layer.11.attention.self.query.bias
transformer.encoder.layer.11.attention.self.key.weight
transformer.encoder.layer.11.attention.self.key.bias
transformer.encoder.layer.11.attention.self.value.weight
transformer.encoder.layer.11.attention.self.value.bias
transformer.encoder.layer.11.attention.output.dense.weight
transformer.encoder.layer.11.attention.output.dense.bias
transformer.encoder.layer.11.attention.output.LayerNorm.weight
transformer.encoder.layer.11.attention.output.LayerNorm.bias
transformer.encoder.layer.11.intermediate.dense.weight
transformer.encoder.layer.11.intermediate.dense.bias
transformer.encoder.layer.11.output.dense.weight
transformer.encoder.layer.11.output.dense.bias
transformer.encoder.layer.11.output.LayerNorm.weight
transformer.encoder.layer.11.output.LayerNorm.bias
transformer.pooler.dense.weight
transformer.pooler.dense.bias
classifier.0.weight
classifier.0.bias
classifier.3.weight
classifier.3.bias

Basically, we want to fine-tune the last layer of the transformer and the classifier
"""
for name, param in model.named_parameters():
    if 'transformer' in name:
        if'encoder' in name and 'layer.11' in name:
            param.requires_grad = True
        elif 'pooler' in name:
            param.requires_grad = True
        else:
            param.requires_grad = False
    elif 'classifier' in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

# Check if all parameters are frozen except for the ones we want
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)


transformer.encoder.layer.11.attention.self.query.weight
transformer.encoder.layer.11.attention.self.query.bias
transformer.encoder.layer.11.attention.self.key.weight
transformer.encoder.layer.11.attention.self.key.bias
transformer.encoder.layer.11.attention.self.value.weight
transformer.encoder.layer.11.attention.self.value.bias
transformer.encoder.layer.11.attention.output.dense.weight
transformer.encoder.layer.11.attention.output.dense.bias
transformer.encoder.layer.11.attention.output.LayerNorm.weight
transformer.encoder.layer.11.attention.output.LayerNorm.bias
transformer.encoder.layer.11.intermediate.dense.weight
transformer.encoder.layer.11.intermediate.dense.bias
transformer.encoder.layer.11.output.dense.weight
transformer.encoder.layer.11.output.dense.bias
transformer.encoder.layer.11.output.LayerNorm.weight
transformer.encoder.layer.11.output.LayerNorm.bias
transformer.pooler.dense.weight
transformer.pooler.dense.bias
classifier.0.weight
classifier.0.bias
classifier.3.weight


In [7]:
# loss and optimizer
from torch import optim
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# training loop
from tqdm import tqdm
from tqdm.notebook import tqdm as tqdm_notebook

desc = f'Epoch {1} loss: ?? Avg shape: ??'
epochs = 20
model.train()
for epoch in range(epochs):
    running_loss = 0.0
    shapes = []
    pbar2 = tqdm(enumerate(dataloader), leave=True, total=len(dataloader), desc=desc)
    for i, data in pbar2:
        # get the inputs; data is a list of [inputs, labels]
        candidates_input_ids, candidates_attention_mask, candidates_token_type_ids, contexts_input_ids, contexts_attention_mask, contexts_token_type_ids, full_mentions_input_ids, full_mentions_attention_mask, full_mentions_token_type_ids, labels = data
        shapes.append(candidates_input_ids.shape[0])
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = model(candidates_input_ids, candidates_attention_mask, candidates_token_type_ids, contexts_input_ids, contexts_attention_mask, contexts_token_type_ids, full_mentions_input_ids, full_mentions_attention_mask, full_mentions_token_type_ids)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # print statistics
        running_loss += loss.item()
        pbar2.update(1)
    desc = f'Epoch {epoch+1} loss: {running_loss / len(dataloader)} Avg shape: {sum(shapes) / len(shapes)}'


Epoch 1 loss: ?? Avg shape: ??:   0%|          | 0/1143 [00:00<?, ?it/s]

Epoch 1 loss: ?? Avg shape: ??:   0%|          | 5/1143 [00:14<53:21,  2.81s/it]


KeyboardInterrupt: 

In [ ]:
query_embeddings 

if isinstance(query_embeddings, (np.ndarray, np.generic)):
    query_embeddings = torch.from_numpy(query_embeddings)
elif isinstance(query_embeddings, list):
    query_embeddings = torch.stack(query_embeddings)

if len(query_embeddings.shape) == 1:
    query_embeddings = query_embeddings.unsqueeze(0)

if isinstance(corpus_embeddings, (np.ndarray, np.generic)):
    corpus_embeddings = torch.from_numpy(corpus_embeddings)
elif isinstance(corpus_embeddings, list):
    corpus_embeddings = torch.stack(corpus_embeddings)

In [ ]:
# let's do predictions on the test set
model.eval()
with torch.no_grad():
    predictions = []
    prediction_indexes = []
    for i, data in tqdm(enumerate(test_dataloader)):
        document_embeds, sentence_embeddings, entity_embeddings, candidate_ids, candidate_description_embeddings = data
        outputs = model((document_embeds, sentence_embeddings, entity_embeddings, candidate_ids, candidate_description_embeddings))
        # which index has the highest value?
        _, predicted = torch.max(outputs, 1)
        prediction_indexes.extend(predicted.tolist())
        # predicted is shape (batch_size)
        # we want to grab the best candidate for batch item i
        # i.e the candidate_ids is of shape (batch_size, 5)
        # we want to grab the best candidate for batch item i
        # i.e get it to shape (batch_size)
        best_candidates = candidate_ids[torch.arange(candidate_ids.size(0)), predicted]

        # Append the best candidates to the predictions list
        predictions.extend(best_candidates.tolist())
        

144it [00:09, 14.82it/s]


In [ ]:
# predictions_from_statistics = []

# for i, row in tqdm(test_dataset.entity_df.iterrows()):
#     full_mention = row['full_mention'].strip().lower()
#     if full_mention in pytorch_utils.anchor_to_candidate:
#         best_candidate = pytorch_utils.anchor_to_candidate[full_mention][0]
#     else:
#         best_candidate = 0
#     predictions_from_statistics.append(best_candidate)

In [ ]:
wiki_items = pd.read_csv(DATA_DIR + 'wiki_lite/wiki_items.csv')
# index wiki_items by id
wiki_items = wiki_items.set_index('item_id')
# Create item_id to wikipedia_title map
item_id_to_title = wiki_items['wikipedia_title'].to_dict()

In [ ]:
enwiki_redirects = pd.read_csv(DATA_DIR + 'wiki_lite/enwiki_redirects.tsv', sep='\t', header=None, names=['source', 'target'])
# index enwiki_redirects by source
enwiki_redirects = enwiki_redirects.set_index('source')
# create source to target map
source_to_target = enwiki_redirects['target'].to_dict()

In [ ]:
wiki_urls = []
not_found = 0
found = 0
redirection = 0
# Now we will map these into wikipedia_urls
for i in tqdm(range(len(predictions))):
    if predictions[i] == 0:
        # if the prediction is 0, we will append a blank url
        wiki_urls.append('NOT_FOUND')
        not_found += 1
        continue
    wikipedia_title = item_id_to_title[predictions[i]]
    # does this wikipedia title exist in the redirects?
    if wikipedia_title in source_to_target:
        # if it does, we will replace it with the redirect
        wikipedia_title = source_to_target[wikipedia_title]
        redirection += 1
    # Now replace the spaces with underscores
    wikipedia_title = wikipedia_title.replace(' ', '_')
    # And add the wikipedia url
    wiki_urls.append(f'http://en.wikipedia.org/wiki/{wikipedia_title}')
    found += 1

print(f'Found {found} wikipedia urls')
print(f'Not found {not_found} wikipedia urls')
print(f'Percentage of wikipedia urls found: {found / (found + not_found)}')
print(f'Percentage of wikipedia urls redirected: {redirection / found}')

100%|██████████| 9166/9166 [00:00<00:00, 1663205.30it/s]

Found 8803 wikipedia urls
Not found 363 wikipedia urls
Percentage of wikipedia urls found: 0.9603971197905302
Percentage of wikipedia urls redirected: 0.0029535385663978187


In [ ]:
# not_found = 0
# found = 0
# train_wiki_urls = []
# # Now we will map these into wikipedia_urls
# for i in tqdm(range(len(predictions_train))):
#     if predictions_train[i] == 0:
#         # if the prediction is 0, we will append a blank url
#         train_wiki_urls.append('NOT_FOUND')
#         not_found += 1
#         continue
#     wikipedia_title = item_id_to_title[predictions_train[i]]
#     # does this wikipedia title exist in the redirects?
#     if wikipedia_title in source_to_target:
#         # if it does, we will replace it with the redirect
#         new_title = source_to_target[wikipedia_title]
#     # Now replace the spaces with underscores
#     wikipedia_title = wikipedia_title.replace(' ', '_')
#     # And add the wikipedia url
#     train_wiki_urls.append(f'http://en.wikipedia.org/wiki/{wikipedia_title}')
#     found += 1

# print(f'Found {found} wikipedia urls')
# print(f'Not found {not_found} wikipedia urls')
# print(f'Percentage of wikipedia urls found: {found / (found + not_found)}')

In [ ]:
test = pd.read_csv(DATA_DIR + 'test.csv')
# train = pd.read_csv(DATA_DIR + 'train.csv')

In [ ]:
not_nan = test['wiki_url'].notna()
not_nme = test['wiki_url'] != '--NME--'
# train_not_nan = train['wiki_url'].notna()
# train_not_nme = train['wiki_url'] != '--NME--'
test.loc[(not_nan & not_nme) & (test.id == 65002)]

,id,token,entity_tag,full_mention,wiki_url
65002,65002,Dejan,B,Dejan Koturovic,?


In [ ]:
test.loc[not_nan & not_nme, 'wiki_url'] = wiki_urls
# train.loc[train_not_nan & train_not_nme, 'wiki_url'] = train_wiki_urls

In [ ]:
# replace NaN or --NME-- with NOT_FOUND
test['wiki_url'] = test['wiki_url'].fillna('NOT_FOUND')
test['wiki_url'] = test['wiki_url'].replace('--NME--', 'NOT_FOUND')
# train['wiki_url'] = train['wiki_url'].fillna('NOT_FOUND')
# train['wiki_url'] = train['wiki_url'].replace('--NME--', 'NOT_FOUND')

In [ ]:
test[test.id == 65002]

,id,token,entity_tag,full_mention,wiki_url
65002,65002,Dejan,B,Dejan Koturovic,http://en.wikipedia.org/wiki/Dejan_Koturović


In [ ]:
' '.join(test.token.fillna('', inplace=False).to_list()[:250])

"-DOCSTART- (947testa CRICKET) CRICKET - LEICESTERSHIRE TAKE OVER AT TOP AFTER INNINGS VICTORY .  LONDON 1996-08-30  West Indian all-rounder Phil Simmons took four for 38 on Friday as Leicestershire beat Somerset by an innings and 39 runs in two days to take over at the head of the county championship .  Their stay on top , though , may be short-lived as title rivals Essex , Derbyshire and Surrey all closed in on victory while Kent made up for lost time in their rain-affected match against Nottinghamshire .  After bowling Somerset out for 83 on the opening morning at Grace Road , Leicestershire extended their first innings by 94 runs before being bowled out for 296 with England discard Andy Caddick taking three for 83 .  Trailing by 213 , Somerset got a solid start to their second innings before Simmons stepped in to bundle them out for 174 .  Essex , however , look certain to regain their top spot after Nasser Hussain and Peter Such gave them a firm grip on their match against Yorkshi

In [ ]:
# # One problem, Simmons does not retrieve the cricket player, but querying for Phil Simmons does.
# # TODO: Fix this, one idea is to group by per doc id, check if this token has a better previous full mention
# display(wiki_items[wiki_items.index.isin(test_dataset.anchor_to_candidate['simmons'])])

# print('____')

# display(wiki_items[wiki_items.index.isin(test_dataset.anchor_to_candidate['phil simmons'])])

In [ ]:
# now create a .csv file from id, wiki_url
test[['id', 'wiki_url']].to_csv('submission_with_doc_lower_score_limit_sample_skip.csv', index=False)
# train[['id', 'wiki_url']].to_csv('train_with_doc.csv', index=False)